In [1]:
%cd /Midgard/home/amirme/an_explanation_model

/Midgard/home/amirme/an_explanation_model


In [18]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
from functools import partial

from source.project_manager import (
    load_experiment_inconsistency,
    load_experiment_metadata,
    delete_experiment_data,
    check_file_exists,
)
from commands.experiment_6_0 import (
    save_metadata_dir,
    dataset_dir,
    input_shape,
    save_output_dir,
)
os.makedirs(save_output_dir, exist_ok=True)
from source.data_manager import (
    preprocess_masks,
    plot_masks,
    minmax_normalize,
    sum_channels,
    symmetric_minmax_normalize,
    single_query_imagenet,
    save_axis,
)
from source.labels import IMAGENET_LABELS

imshow_args = {"cmap": "hsv"}

query_imagenet = partial(
    single_query_imagenet, dataset_dir=dataset_dir, input_shape=input_shape
)

In [3]:
project_metadata = load_experiment_metadata(save_metadata_dir)
print(project_metadata.columns)
project_metadata = project_metadata.set_index(
    [
        "stream_name",
        "stream_statistic",
        "image_index",
        "baseline_mask_type",
        "alpha_mask_value",
    ]
).sort_index()
print(project_metadata.shape)

Index(['time_to_compute', 'batch_index', 'monitored_statistic_change',
       'path_prefix', 'data_path', 'stream_name', 'stream_statistic',
       'alpha_mask_type', 'alpha_mask_value', 'baseline_mask_type',
       'projection_type', 'projection_distribution', 'projection_top_k',
       'projection_index', 'label', 'architecture', 'method', 'output_layer',
       'dataset', 'image_index', 'input_shape', 'num_classes', 'seed',
       'batch_size', 'max_batches', 'min_change', 'metadata_file_path'],
      dtype='object')
(2100, 22)


In [4]:
vc = project_metadata.index.value_counts()
print(vc)
print(len(vc))
image_indices = project_metadata.index.get_level_values("image_index").unique()

stream_name        stream_statistic  image_index  baseline_mask_type  alpha_mask_value
log_probs          meanx             1            gaussian            0.0                 1
vanilla_grad_mask  meanx2            2            gaussian            0.4                 1
                                                                      0.2                 1
                                                                      0.1                 1
                                                                      0.0                 1
                                                                                         ..
log_probs          meanx             100          gaussian            0.1                 1
                                                                      0.0                 1
                                     99           gaussian            0.6                 1
                                                                      0.5            

In [5]:
for name in project_metadata.index.names:
    print(name,"=",project_metadata.index.get_level_values(name).value_counts().to_dict())

stream_name = {'log_probs': 700, 'results_at_projection': 700, 'vanilla_grad_mask': 700}
stream_statistic = {'meanx': 1400, 'meanx2': 700}
image_index = {1: 21, 64: 21, 74: 21, 73: 21, 72: 21, 71: 21, 70: 21, 69: 21, 68: 21, 67: 21, 66: 21, 65: 21, 63: 21, 2: 21, 62: 21, 61: 21, 60: 21, 59: 21, 58: 21, 57: 21, 56: 21, 55: 21, 54: 21, 53: 21, 75: 21, 76: 21, 77: 21, 78: 21, 99: 21, 98: 21, 97: 21, 96: 21, 95: 21, 94: 21, 93: 21, 92: 21, 91: 21, 90: 21, 89: 21, 88: 21, 87: 21, 86: 21, 85: 21, 84: 21, 83: 21, 82: 21, 81: 21, 80: 21, 79: 21, 52: 21, 51: 21, 50: 21, 25: 21, 23: 21, 22: 21, 21: 21, 20: 21, 19: 21, 18: 21, 17: 21, 16: 21, 15: 21, 14: 21, 13: 21, 12: 21, 11: 21, 10: 21, 9: 21, 8: 21, 7: 21, 6: 21, 5: 21, 4: 21, 3: 21, 24: 21, 26: 21, 49: 21, 27: 21, 48: 21, 47: 21, 46: 21, 45: 21, 44: 21, 43: 21, 42: 21, 41: 21, 40: 21, 39: 21, 38: 21, 37: 21, 36: 21, 35: 21, 34: 21, 33: 21, 32: 21, 31: 21, 30: 21, 29: 21, 28: 21, 100: 21}
baseline_mask_type = {'gaussian': 2100}
alpha_mask_val

In [16]:
def spectral_lens(image,eps=1e-3):
    amplitude = 1/(image["alpha_mask_value"].values+eps)
    image = np.stack(image["grad_mask"].values)
    amplitude= np.expand_dims(amplitude,axis=(1,2,3))
    assert image.ndim==4, f"{image.shape} must be 4d (T,C,H,W)"
    assert amplitude.ndim==4, f"{amplitude.shape} must be 4d (T,1,1,1)"
    image = image*amplitude
    image = image.sum(axis=0)
    return image

In [17]:
image_indices = [7,14,41,20]
explanations_temp = project_metadata.loc[("vanilla_grad_mask", "meanx2", image_indices, "gaussian"), "data_path"]
explanations_temp = explanations_temp.droplevel(["stream_name","stream_statistic","baseline_mask_type"])
explanations_temp = explanations_temp.apply(np.load)
explanations_temp.name = "grad_mask"
explanations_temp = explanations_temp.reset_index()
explanations_temp = explanations_temp.groupby("image_index",as_index=False).apply(lambda x:spectral_lens(x))

In [ ]:
n_cols=4
fig,axes = plt.subplots(image_indices//n_cols,n_cols,figsize=(15*n_cols,15))
for ax,image_index in zip(axes.ravel(),image_indices):
    image = explanations_temp.loc[image_index]
    ax.imshow(image,**imshow_args)
    ax.set_title(f"image_index={image_index}")

plt.show()